# The NN with baeysian regulated backpropagation in matlab with missing data: complete case & Kth nearest imputation case analysis 

# The main code is based on the book "practical-machine-learning-with-python" and its author's github "https://github.com/dipanjanS/practical-machine-learning-with-python.git"

In [58]:
import pandas as pd
import math
import os
import fancyimpute
from sklearn.preprocessing import Imputer  
from fancyimpute import KNN
import numpy as np
import re

In [24]:
# turn of warning messages
pd.options.mode.chained_assignment = None  # default='warn'

# get data
df = pd.read_csv('train_ajEneEa.csv')
df.columns
df = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke']]
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
5,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,NaN,0
6,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
8,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
9,Female,74.0,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0


In [25]:
# Data description
print("Number of rows::",df.shape[0])
print("Number of columns::",df.shape[1] )
print("Column Names::",df.columns.values.tolist())
print("Column Data Types::\n",df.dtypes)
print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
print("General Stats::")
print(df.info())
print("Summary Stats::" )
print(df.describe())

# Missing data --> Columns with Missing Values:: ['bmi', 'smoking_status']
#                  Number of rows with Missing Values:: 14328

Number of rows:: 43400
Number of columns:: 11
Column Names:: ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']
Column Data Types::
 gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object
Columns with Missing Values:: ['bmi', 'smoking_status']
Number of rows with Missing Values:: 14328
General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43400 entries, 0 to 43399
Data columns (total 11 columns):
gender               43400 non-null object
age                  43400 non-null float64
hypertension         43400 non-null int64
heart_disease        43400 non-null int64
ever_married         43400 non-null 

In [26]:
#remove missing values/rows
#Columns with Missing Values:: ['bmi', 'smoking_status']
print("Drop Rows with missing bmi::" )
df_dropped = df.dropna(subset=['bmi'])
print("Shape::",df_dropped.shape)

Drop Rows with missing bmi::
Shape:: (41938, 11)


In [27]:
# remove missing rows associated to smoking_status
print("Drop Rows with missing smoking_status::" )
df_dropped = df_dropped.dropna(subset=['smoking_status'])
print("Shape::",df_dropped.shape)

Drop Rows with missing smoking_status::
Shape:: (29072, 11)


In [28]:
# check there is no more missing row/columns
print("Columns with Missing Values::",df_dropped.columns[df.isnull().any()].tolist())
print("Number of rows with Missing Values::",len(pd.isnull(df_dropped).any(1).nonzero()[0].tolist()))

Columns with Missing Values:: ['bmi', 'smoking_status']
Number of rows with Missing Values:: 0


# Another way of handing missing data
# Kth nearest imputation

In [29]:
# turn of warning messages
pd.options.mode.chained_assignment = None  # default='warn'

# get data
df = pd.read_csv('train_ajEneEa.csv')
df.columns
df = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke']]
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
5,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,NaN,0
6,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
8,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
9,Female,74.0,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0


In [30]:
#first obtain all of the factor levels of predictors
df.columns
print(pd.Series.unique(df.smoking_status))
print(pd.Series.unique(df.gender) )
print(pd.Series.unique(df.ever_married))
print(pd.Series.unique(df.work_type))
print(pd.Series.unique(df.Residence_type))

[nan 'never smoked' 'formerly smoked' 'smokes']
['Male' 'Female' 'Other']
['No' 'Yes']
['children' 'Private' 'Never_worked' 'Self-employed' 'Govt_job']
['Rural' 'Urban']


In [31]:
#change string to numeric for columns with missing value
# to use fancyimpute
df.smoking_status=df.smoking_status.map({'never smoked':1, 'formerly smoked':0, 'smokes':-1})
df.gender=df.gender.map({'Male':1, 'Female':0, 'Other':-1})
df.ever_married = df.ever_married.map({'No':1 ,'Yes':0})
df.work_type = df.work_type.map({'children':1 , 'Private':2, 'Never_worked':3, 'Self-employed':4, 'Govt_job':5})
df.Residence_type = df.Residence_type.map({'Rural':1, 'Urban':0})
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,3.0,0,0,1,1,1,95.12,18.0,NaN,0
1,1,58.0,1,0,0,2,0,87.96,39.2,1.0,0
2,0,8.0,0,0,1,2,0,110.89,17.6,NaN,0
3,0,70.0,0,0,0,2,1,69.04,35.9,0.0,0
4,1,14.0,0,0,1,3,1,161.28,19.1,NaN,0
5,0,47.0,0,0,0,2,0,210.95,50.1,NaN,0
6,0,52.0,0,0,0,2,0,77.59,17.7,0.0,0
7,0,75.0,0,1,0,4,1,243.53,27.0,1.0,0
8,0,32.0,0,0,0,2,1,77.67,32.3,-1.0,0
9,0,74.0,1,0,0,4,0,205.84,54.6,1.0,0


In [33]:
# change the value "K" as you want
df_kth = fancyimpute.KNN(k=3,orientation='columns').complete(df)

Imputing row 1/11 with 0 missing, elapsed time: 0.059


In [34]:
# reconstruct the data with the imputing result
values = df_kth
index = ['Row'+str(i) for i in range(1, len(values)+1)]

# Let dftest refer to the Kth nearest imputed data
dftest = pd.DataFrame(values, index=index)

In [16]:
dftest

,0,1,2,3,4,5,6,7,8,9,10
Row1,1.0,3.0,0.0,0.0,1.0,1.0,1.0,95.12,18.0,0.0,0.0
Row2,1.0,58.0,1.0,0.0,0.0,2.0,0.0,87.96,39.2,1.0,0.0
Row3,0.0,8.0,0.0,0.0,1.0,2.0,0.0,110.89,17.6,0.0,0.0
Row4,0.0,70.0,0.0,0.0,0.0,2.0,1.0,69.04,35.9,0.0,0.0
Row5,1.0,14.0,0.0,0.0,1.0,3.0,1.0,161.28,19.1,0.0,0.0
Row6,0.0,47.0,0.0,0.0,0.0,2.0,0.0,210.95,50.1,0.0,0.0
Row7,0.0,52.0,0.0,0.0,0.0,2.0,0.0,77.59,17.7,0.0,0.0
Row8,0.0,75.0,0.0,1.0,0.0,4.0,1.0,243.53,27.0,1.0,0.0
Row9,0.0,32.0,0.0,0.0,0.0,2.0,1.0,77.67,32.3,-1.0,0.0
Row10,0.0,74.0,1.0,0.0,0.0,4.0,0.0,205.84,54.6,1.0,0.0


In [35]:
# Reconstruct the collumn names 
dftest.columns = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke']

In [36]:
dftest

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
Row1,1.0,3.0,0.0,0.0,1.0,1.0,1.0,95.12,18.000000,0.328775,0.0
Row2,1.0,58.0,1.0,0.0,0.0,2.0,0.0,87.96,39.200000,1.000000,0.0
Row3,0.0,8.0,0.0,0.0,1.0,2.0,0.0,110.89,17.600000,0.328775,0.0
Row4,0.0,70.0,0.0,0.0,0.0,2.0,1.0,69.04,35.900000,0.000000,0.0
Row5,1.0,14.0,0.0,0.0,1.0,3.0,1.0,161.28,19.100000,0.328775,0.0
Row6,0.0,47.0,0.0,0.0,0.0,2.0,0.0,210.95,50.100000,0.000000,0.0
Row7,0.0,52.0,0.0,0.0,0.0,2.0,0.0,77.59,17.700000,0.000000,0.0
Row8,0.0,75.0,0.0,1.0,0.0,4.0,1.0,243.53,27.000000,1.000000,0.0
Row9,0.0,32.0,0.0,0.0,0.0,2.0,1.0,77.67,32.300000,-1.000000,0.0
Row10,0.0,74.0,1.0,0.0,0.0,4.0,0.0,205.84,54.600000,1.000000,0.0


In [37]:
# The numericalized categorical variables need to be back to 
# the original categorical values         --- smoking status ---

# Remember we only assign 1, 0, -1 For smoking_status
# we take take a rounded value to decided a factor level of 
# imputed data
print(pd.Series.unique(dftest.smoking_status) )
dftest.smoking_status = dftest.smoking_status.astype(int)

[ 0.32877461  1.          0.         -1.          0.32854112  0.34268424
  0.67122536  0.99999997  0.67145885  0.65731573]


In [40]:
print(pd.Series.unique(dftest.smoking_status))

[ 0  1 -1]


In [41]:
#Get numericalized values be original categorical values
#change string to numeric for columns with missing value -- others
dftest.smoking_status=dftest.smoking_status.map({1:'never smoked', 0:'formerly smoked', -1:'smokes'})
dftest.gender=dftest.gender.map({1:'Male', 0:'Female', -1:'Other'})
dftest.ever_married = dftest.ever_married.map({1:'No' ,0:'Yes'})
dftest.work_type = dftest.work_type.map({1:'children',2: 'Private', 3:'Never_worked', 4:'Self-employed', 5:'Govt_job'})
dftest.Residence_type = dftest.Residence_type.map({1.0:'Rural', 0.0:'Urban'})

In [42]:
# Regarding stroke, float --> int
dftest.stroke = dftest.stroke.astype(int)
dftest

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
Row1,Male,3.0,0.0,0.0,No,children,Rural,95.12,18.000000,formerly smoked,0
Row2,Male,58.0,1.0,0.0,Yes,Private,Urban,87.96,39.200000,never smoked,0
Row3,Female,8.0,0.0,0.0,No,Private,Urban,110.89,17.600000,formerly smoked,0
Row4,Female,70.0,0.0,0.0,Yes,Private,Rural,69.04,35.900000,formerly smoked,0
Row5,Male,14.0,0.0,0.0,No,Never_worked,Rural,161.28,19.100000,formerly smoked,0
Row6,Female,47.0,0.0,0.0,Yes,Private,Urban,210.95,50.100000,formerly smoked,0
Row7,Female,52.0,0.0,0.0,Yes,Private,Urban,77.59,17.700000,formerly smoked,0
Row8,Female,75.0,0.0,1.0,Yes,Self-employed,Rural,243.53,27.000000,never smoked,0
Row9,Female,32.0,0.0,0.0,Yes,Private,Rural,77.67,32.300000,smokes,0
Row10,Female,74.0,1.0,0.0,Yes,Self-employed,Urban,205.84,54.600000,never smoked,0


In [44]:
#Final check for NA after imputation
print("Columns with Missing Values::",dftest.columns[dftest.isnull().any()].tolist())
print("Number of rows with Missing Values::",len(pd.isnull(dftest).any(1).nonzero()[0].tolist()))

Columns with Missing Values:: []
Number of rows with Missing Values:: 0


## Creating a set of predictors and target responses 

In [45]:
#--- IF YOU WANT TO IGNORE WHOLE NA VALUES COMMENT OUT THE FOLLOWING LINE--- #
df = dftest
# get features and corresponding outcomes
feature_names = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status']
training_features = df[feature_names]

outcome_name = ['stroke']
outcome_labels = df[outcome_name]

In [46]:
training_features

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
Row1,Male,3.0,0.0,0.0,No,children,Rural,95.12,18.000000,formerly smoked
Row2,Male,58.0,1.0,0.0,Yes,Private,Urban,87.96,39.200000,never smoked
Row3,Female,8.0,0.0,0.0,No,Private,Urban,110.89,17.600000,formerly smoked
Row4,Female,70.0,0.0,0.0,Yes,Private,Rural,69.04,35.900000,formerly smoked
Row5,Male,14.0,0.0,0.0,No,Never_worked,Rural,161.28,19.100000,formerly smoked
Row6,Female,47.0,0.0,0.0,Yes,Private,Urban,210.95,50.100000,formerly smoked
Row7,Female,52.0,0.0,0.0,Yes,Private,Urban,77.59,17.700000,formerly smoked
Row8,Female,75.0,0.0,1.0,Yes,Self-employed,Rural,243.53,27.000000,never smoked
Row9,Female,32.0,0.0,0.0,Yes,Private,Rural,77.67,32.300000,smokes
Row10,Female,74.0,1.0,0.0,Yes,Self-employed,Urban,205.84,54.600000,never smoked


In [47]:
numeric_feature_names = ['age', 'hypertension', 'heart_disease',  'avg_glucose_level', 'bmi']
categoricial_feature_names = ['gender', 'ever_married','work_type', 'Residence_type','smoking_status']

In [48]:
#Standardized data (especially important for Neural net usage..?)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(training_features[numeric_feature_names])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [49]:
#mean..
ss.mean_

array([4.22178940e+01, 9.35714286e-02, 4.75115207e-02, 1.04482750e+02,
       2.83750363e+01])

In [50]:
#variance
ss.var_

array([5.07122892e+02, 8.48158163e-02, 4.52541761e-02, 1.85858024e+03,
       6.21572163e+01])

In [51]:
# scale numeric features now--Perform standardization by centering and scaling
training_features[numeric_feature_names] = ss.transform(training_features[numeric_feature_names])

# view updated featureset
training_features

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
Row1,Male,-1.741517,-0.321296,-0.223342,No,children,Rural,-0.217176,-1.315964,formerly smoked
Row2,Male,0.700823,3.112398,-0.223342,Yes,Private,Urban,-0.383258,1.373032,never smoked
Row3,Female,-1.519486,-0.321296,-0.223342,No,Private,Urban,0.148621,-1.366699,formerly smoked
Row4,Female,1.233697,-0.321296,-0.223342,Yes,Private,Rural,-0.822123,0.954462,formerly smoked
Row5,Male,-1.253049,-0.321296,-0.223342,No,Never_worked,Rural,1.317458,-1.176440,formerly smoked
Row6,Female,0.212355,-0.321296,-0.223342,Yes,Private,Urban,2.469593,2.755582,formerly smoked
Row7,Female,0.434386,-0.321296,-0.223342,Yes,Private,Urban,-0.623799,-1.354015,formerly smoked
Row8,Female,1.455728,-0.321296,4.477446,Yes,Self-employed,Rural,3.225312,-0.174409,never smoked
Row9,Female,-0.453738,-0.321296,-0.223342,Yes,Private,Rural,-0.621943,0.497840,smokes
Row10,Female,1.411322,3.112398,-0.223342,Yes,Self-employed,Urban,2.351062,3.326359,never smoked


In [52]:
# creating dummy variables for categorical variables
training_features = pd.get_dummies(training_features, columns=categoricial_feature_names)
# view newly engineering features
training_features

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
Row1,-1.741517,-0.321296,-0.223342,-0.217176,-1.315964,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0
Row2,0.700823,3.112398,-0.223342,-0.383258,1.373032,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0
Row3,-1.519486,-0.321296,-0.223342,0.148621,-1.366699,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0
Row4,1.233697,-0.321296,-0.223342,-0.822123,0.954462,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0
Row5,-1.253049,-0.321296,-0.223342,1.317458,-1.176440,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0
Row6,0.212355,-0.321296,-0.223342,2.469593,2.755582,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0
Row7,0.434386,-0.321296,-0.223342,-0.623799,-1.354015,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0
Row8,1.455728,-0.321296,4.477446,3.225312,-0.174409,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0
Row9,-0.453738,-0.321296,-0.223342,-0.621943,0.497840,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1
Row10,1.411322,3.112398,-0.223342,2.351062,3.326359,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0


In [53]:
# get list of new categorical features
categorical_engineered_features = list(set(training_features.columns) - set(numeric_feature_names))

In [54]:
categorical_engineered_features

['Residence_type_Rural',
 'Residence_type_Urban',
 'gender_Female',
 'work_type_Self-employed',
 'gender_Other',
 'work_type_Govt_job',
 'smoking_status_formerly smoked',
 'smoking_status_never smoked',
 'smoking_status_smokes',
 'gender_Male',
 'work_type_Never_worked',
 'ever_married_No',
 'work_type_children',
 'ever_married_Yes',
 'work_type_Private']

# # Creating .mat datafile for matlab

In [55]:
import scipy
from scipy import io

### 3rd nearest imputation for NA cases 

In [ ]:
numpyMatrix_k3th = training_features.as_matrix()
numpyMatrix_target_k3th = outcome_labels.as_matrix()
scipy.io.savemat('K3nearestMissing_train.mat',  mdict={'matrix': numpyMatrix_k3th})
scipy.io.savemat('K3nearestMissing_train_target.mat', {'matrix': numpyMatrix_target_k3th})

### Completely ignoring NA cases

In [ ]:
scipy.io.savemat('NAremove_train.mat',  mdict={'matrix': training_features})

#  # Importing test data set.

In [56]:
#test = training_features.as_matrix
test = training_features.to_dict()

In [57]:
# create predictor and responses for test set
numpyMatrix = training_features.as_matrix()
numpyMatrix_target = outcome_labels.as_matrix()

# # Modling (logistic) + preparing test dataset

In [59]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# fit the model
lr = LogisticRegression() 
model = lr.fit(training_features, np.array(outcome_labels['stroke']))
# view model parameters
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [60]:
# simple evaluation on training data
pred_labels = model.predict(training_features)
actual_labels = np.array(outcome_labels['stroke'])

# evaluate model performance --- TRAINING SET!!! ----
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('Accuracy:', float(accuracy_score(actual_labels, pred_labels))*100, '%')
print('Classification Stats:')
print(classification_report(actual_labels, pred_labels))

Accuracy: 98.1958525345622 %
Classification Stats:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     42617
          1       0.00      0.00      0.00       783

avg / total       0.96      0.98      0.97     43400



/export/home1/NoCsBack/thesisnl/r0649141/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
from sklearn.externals import joblib
import os
# save models to be deployed on your server
if not os.path.exists('Model'):
    os.mkdir('Model')
if not os.path.exists('Scaler'):
    os.mkdir('Scaler') 
    
joblib.dump(model, r'Model/model.pickle') 
joblib.dump(ss, r'Scaler/scaler.pickle')

In [63]:
# load model and scaler objects
model = joblib.load(r'Model/model.pickle')
scaler = joblib.load(r'Scaler/scaler.pickle')

In [96]:
#prepare TEST SET
# get data
prediction_features = pd.read_csv('test_v2akXPA.csv')
prediction_features

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,NaN
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,NaN
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked
5,9348,Female,66.0,1,0,Yes,Private,Urban,219.98,32.2,never smoked
6,51550,Female,49.0,0,0,Yes,Self-employed,Rural,74.03,25.1,NaN
7,60512,Male,46.0,0,0,Yes,Govt_job,Urban,120.80,32.5,never smoked
8,31309,Female,75.0,0,0,Yes,Self-employed,Rural,78.71,28.0,never smoked
9,39199,Male,75.0,0,0,Yes,Self-employed,Urban,77.20,25.7,smokes


In [97]:
prediction_features.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status'],
      dtype='object')

In [98]:
# To create predictors, drop 'id'
prediction_features = prediction_features[[ 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status']]

In [99]:
# Descriptive Data 
print("Number of rows::",prediction_features.shape[0])
print("Number of columns::",prediction_features.shape[1] )
print("Column Names::",prediction_features.columns.values.tolist())
print("Column Data Types::\n",prediction_features.dtypes)
print("Columns with Missing Values::",prediction_features.columns[prediction_features.isnull().any()].tolist())
print("Number of rows with Missing Values::",len(pd.isnull(prediction_features).any(1).nonzero()[0].tolist()))
print("General Stats::")
print(prediction_features.info())
print("Summary Stats::" )
print(prediction_features.describe())
#IN TOTAL 1/3 OF ROWS INCLUDES MISSING VALLUES..

Number of rows:: 18601
Number of columns:: 10
Column Names:: ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']
Column Data Types::
 gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
dtype: object
Columns with Missing Values:: ['bmi', 'smoking_status']
Number of rows with Missing Values:: 6178
General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18601 entries, 0 to 18600
Data columns (total 10 columns):
gender               18601 non-null object
age                  18601 non-null float64
hypertension         18601 non-null int64
heart_disease        18601 non-null int64
ever_married         18601 non-null object
work_type            18601 non-nu

### Missing data handling for TEST DATA SET (Kth nearest imputation) 

In [69]:
print(pd.Series.unique(prediction_features.gender) )
print(pd.Series.unique(prediction_features.ever_married))
print(pd.Series.unique(prediction_features.work_type))
print(pd.Series.unique(prediction_features.Residence_type))

['Male' 'Female' 'Other']
['Yes' 'No']
['Private' 'Self-employed' 'children' 'Govt_job' 'Never_worked']
['Urban' 'Rural']


In [100]:
#change string to numeric for columns with missing value
prediction_features.smoking_status=prediction_features.smoking_status.map({'never smoked':1, 'formerly smoked':0, 'smokes':-1})
prediction_features.gender=prediction_features.gender.map({'Male':1, 'Female':0, 'Other':-1})
prediction_features.ever_married = prediction_features.ever_married.map({'No':1 ,'Yes':0})
prediction_features.work_type = prediction_features.work_type.map({'children':1 , 'Private':2, 'Never_worked':3, 'Self-employed':4, 'Govt_job':5})
prediction_features.Residence_type = prediction_features.Residence_type.map({'Rural':1, 'Urban':0})
prediction_features

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,80.0,0,0,0,2,0,83.84,21.1,0.0
1,0,74.0,0,1,0,4,1,179.50,26.0,0.0
2,0,14.0,0,0,1,1,1,95.16,21.2,NaN
3,1,28.0,0,0,1,2,0,94.76,23.4,NaN
4,0,63.0,0,0,0,5,1,83.57,27.6,1.0
5,0,66.0,1,0,0,2,0,219.98,32.2,1.0
6,0,49.0,0,0,0,4,1,74.03,25.1,NaN
7,1,46.0,0,0,0,5,0,120.80,32.5,1.0
8,0,75.0,0,0,0,4,1,78.71,28.0,1.0
9,1,75.0,0,0,0,4,0,77.20,25.7,-1.0


In [101]:
df_kth_test = fancyimpute.KNN(k=3,orientation='columns').complete(prediction_features)

Imputing row 1/10 with 0 missing, elapsed time: 0.014


In [102]:
values = df_kth_test
index = ['Row'+str(i) for i in range(1, len(values)+1)]

dftest_test = pd.DataFrame(values, index=index)

In [103]:
dftest_test

,0,1,2,3,4,5,6,7,8,9
Row1,1.0,80.0,0.0,0.0,0.0,2.0,0.0,83.84,21.100000,0.000000
Row2,0.0,74.0,0.0,1.0,0.0,4.0,1.0,179.50,26.000000,0.000000
Row3,0.0,14.0,0.0,0.0,1.0,1.0,1.0,95.16,21.200000,0.335782
Row4,1.0,28.0,0.0,0.0,1.0,2.0,0.0,94.76,23.400000,0.335782
Row5,0.0,63.0,0.0,0.0,0.0,5.0,1.0,83.57,27.600000,1.000000
Row6,0.0,66.0,1.0,0.0,0.0,2.0,0.0,219.98,32.200000,1.000000
Row7,0.0,49.0,0.0,0.0,0.0,4.0,1.0,74.03,25.100000,0.000000
Row8,1.0,46.0,0.0,0.0,0.0,5.0,0.0,120.80,32.500000,1.000000
Row9,0.0,75.0,0.0,0.0,0.0,4.0,1.0,78.71,28.000000,1.000000
Row10,1.0,75.0,0.0,0.0,0.0,4.0,0.0,77.20,25.700000,-1.000000


In [104]:
dftest_test.columns = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status']
print(pd.Series.unique(dftest_test.smoking_status) )
dftest_test.smoking_status = dftest_test.smoking_status.astype(int)
#Get numericalized values be original categorical values
#change string to numeric for columns with missing value
dftest_test.smoking_status=dftest_test.smoking_status.map({1:'never smoked', 0:'formerly smoked', -1:'smokes'})
dftest_test.gender=dftest_test.gender.map({1:'Male', 0:'Female', -1:'Other'})
dftest_test.ever_married = dftest_test.ever_married.map({1:'No' ,0:'Yes'})
dftest_test.work_type = dftest_test.work_type.map({1:'children',2: 'Private', 3:'Never_worked', 4:'Self-employed', 5:'Govt_job'})
dftest_test.Residence_type = dftest_test.Residence_type.map({1.0:'Rural', 0.0:'Urban'})

#print(pd.Series.unique(dftest.stroke) )

[ 0.          0.33578233  1.         -1.          0.33171603  0.66421773
  0.33250169  0.66828403  1.00000006  0.66749837]


In [105]:
print(pd.Series.unique(dftest_test.smoking_status) )

['formerly smoked' 'never smoked' 'smokes']


In [106]:
dftest_test

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
Row1,Male,80.0,0.0,0.0,Yes,Private,Urban,83.84,21.100000,formerly smoked
Row2,Female,74.0,0.0,1.0,Yes,Self-employed,Rural,179.50,26.000000,formerly smoked
Row3,Female,14.0,0.0,0.0,No,children,Rural,95.16,21.200000,formerly smoked
Row4,Male,28.0,0.0,0.0,No,Private,Urban,94.76,23.400000,formerly smoked
Row5,Female,63.0,0.0,0.0,Yes,Govt_job,Rural,83.57,27.600000,never smoked
Row6,Female,66.0,1.0,0.0,Yes,Private,Urban,219.98,32.200000,never smoked
Row7,Female,49.0,0.0,0.0,Yes,Self-employed,Rural,74.03,25.100000,formerly smoked
Row8,Male,46.0,0.0,0.0,Yes,Govt_job,Urban,120.80,32.500000,never smoked
Row9,Female,75.0,0.0,0.0,Yes,Self-employed,Rural,78.71,28.000000,never smoked
Row10,Male,75.0,0.0,0.0,Yes,Self-employed,Urban,77.20,25.700000,smokes


In [107]:
prediction_features = dftest_test

In [108]:
from sklearn.preprocessing import StandardScaler
# scaling
prediction_features[numeric_feature_names] = scaler.transform(prediction_features[numeric_feature_names])

# engineering categorical variables --> dummy coding
prediction_features = pd.get_dummies(prediction_features, columns=categoricial_feature_names)

# view feature set
prediction_features

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
Row1,1.677759,-0.321296,-0.223342,-0.478825,-0.922761,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0
Row2,1.411322,-0.321296,4.477446,1.740085,-0.301248,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0
Row3,-1.253049,-0.321296,-0.223342,-0.216249,-0.910077,1,0,0,1,0,0,0,0,0,1,1,0,1,0,0
Row4,-0.631362,-0.321296,-0.223342,-0.225527,-0.631031,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0
Row5,0.922854,-0.321296,-0.223342,-0.485088,-0.098305,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
Row6,1.056072,3.112398,-0.223342,2.679051,0.485156,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0
Row7,0.301167,-0.321296,-0.223342,-0.706376,-0.415404,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0
Row8,0.167949,-0.321296,-0.223342,0.378492,0.523208,0,1,0,0,1,1,0,0,0,0,0,1,0,1,0
Row9,1.455728,-0.321296,-0.223342,-0.597820,-0.047569,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0
Row10,1.455728,-0.321296,-0.223342,-0.632845,-0.339300,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1


In [109]:
prediction_features.columns
len(prediction_features) 
# add missing categorical feature columns
current_categorical_engineered_features = set(prediction_features.columns) - set(numeric_feature_names)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features

In [110]:
missing_features
# Actually we luckily don't have any missing column..

set()

In [111]:
# Check if the predictors are corresponding to the training set
prediction_features.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Female', 'gender_Male', 'gender_Other', 'ever_married_No',
       'ever_married_Yes', 'work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed', 'work_type_children',
       'Residence_type_Rural', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

In [114]:
#compare with predictors for training set
training_features.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Female', 'gender_Male', 'gender_Other', 'ever_married_No',
       'ever_married_Yes', 'work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed', 'work_type_children',
       'Residence_type_Rural', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

#### Just in case where there are some missing predictors... (especially dummy variables)

In [ ]:
    # add missing categorical feature columns
    current_categorical_engineered_features = set(prediction_features.columns) - set(numeric_feature_names)
    missing_features = set(categorical_engineered_features) - current_categorical_engineered_features
    for feature in missing_features:
        # add zeros since feature is absent in these data samples
        prediction_features[feature] = [0] * len(prediction_features) # 'len(prediction_features) ' takes #new samples. 

    # view final feature set
    prediction_features

#### Just in case you want to export the test data as .mat file...

In [ ]:
numpyMatrix_prediction = prediction_features.as_matrix()
scipy.io.savemat('K3nearestimpute_test.mat', {'matrix': numpyMatrix_prediction})

### Missing data handling for TEST DATA SET (Completely ignore: remove all rows  & columns including NA)  

In [85]:
#Just in case, re-read the original test data
prediction_features = pd.read_csv('test_v2akXPA.csv')
prediction_features

# TRY TO IGNORE MISSING DATAS(REMOVE ROWS WITH MISSING VALUES)
print("Drop Rows with missing smoking_status::" )
prediction_features_dropped = prediction_features.dropna(subset=['smoking_status'])
prediction_features_dropped = prediction_features_dropped.dropna(subset=['bmi'])
print("Shape::",prediction_features_dropped.shape)
print("Columns with Missing Values::",prediction_features_dropped.columns[prediction_features.isnull().any()].tolist())
print("Number of rows with Missing Values::",len(pd.isnull(prediction_features_dropped).any(1).nonzero()[0].tolist()))

Drop Rows with missing smoking_status::
Shape:: (12423, 11)
Columns with Missing Values:: ['bmi', 'smoking_status']
Number of rows with Missing Values:: 0


In [87]:
from sklearn.preprocessing import StandardScaler
# scaling
prediction_features_dropped[numeric_feature_names] = scaler.transform(prediction_features_dropped[numeric_feature_names])

# engineering categorical variables --> dummy coding
prediction_features_dropped = pd.get_dummies(prediction_features_dropped, columns=categoricial_feature_names)

# view feature set
prediction_features_dropped

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,36306,1.677759,-0.321296,-0.223342,-0.478825,-0.922761,0,1,0,1,0,0,1,0,0,0,1,1,0,0
1,61829,1.411322,-0.321296,4.477446,1.740085,-0.301248,1,0,0,1,0,0,0,1,0,1,0,1,0,0
4,40801,0.922854,-0.321296,-0.223342,-0.485088,-0.098305,1,0,0,1,1,0,0,0,0,1,0,0,1,0
5,9348,1.056072,3.112398,-0.223342,2.679051,0.485156,1,0,0,1,0,0,1,0,0,0,1,0,1,0
7,60512,0.167949,-0.321296,-0.223342,0.378492,0.523208,0,1,0,1,1,0,0,0,0,0,1,0,1,0
8,31309,1.455728,-0.321296,-0.223342,-0.597820,-0.047569,1,0,0,1,0,0,0,1,0,1,0,0,1,0
9,39199,1.455728,-0.321296,-0.223342,-0.632845,-0.339300,0,1,0,1,0,0,0,1,0,0,1,0,0,1
12,19042,0.212355,-0.321296,-0.223342,-0.298826,-0.212461,1,0,0,1,0,0,1,0,0,1,0,0,1,0
14,33104,1.100479,-0.321296,-0.223342,3.064565,-0.529559,1,0,0,1,1,0,0,0,0,0,1,0,1,0
15,55264,0.434386,-0.321296,-0.223342,0.116147,-0.491507,1,0,1,0,0,0,0,1,0,0,1,0,1,0


In [88]:
prediction_features_dropped.columns
len(prediction_features_dropped) 
# add missing categorical feature columns
current_categorical_engineered_features = set(prediction_features_dropped.columns) - set(numeric_feature_names)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features

In [89]:
missing_features

{'gender_Other'}

In [90]:
for feature in missing_features:
    # add zeros since feature is absent in these data samples
    prediction_features_dropped[feature] = [0] * len(prediction_features_dropped) # 'len(prediction_features) ' takes #new samples. 

# view final feature set
prediction_features_dropped



,id,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,gender_Other
0,36306,1.677759,-0.321296,-0.223342,-0.478825,-0.922761,0,1,0,1,...,0,1,0,0,0,1,1,0,0,0
1,61829,1.411322,-0.321296,4.477446,1.740085,-0.301248,1,0,0,1,...,0,0,1,0,1,0,1,0,0,0
4,40801,0.922854,-0.321296,-0.223342,-0.485088,-0.098305,1,0,0,1,...,0,0,0,0,1,0,0,1,0,0
5,9348,1.056072,3.112398,-0.223342,2.679051,0.485156,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0
7,60512,0.167949,-0.321296,-0.223342,0.378492,0.523208,0,1,0,1,...,0,0,0,0,0,1,0,1,0,0
8,31309,1.455728,-0.321296,-0.223342,-0.597820,-0.047569,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0
9,39199,1.455728,-0.321296,-0.223342,-0.632845,-0.339300,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0
12,19042,0.212355,-0.321296,-0.223342,-0.298826,-0.212461,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
14,33104,1.100479,-0.321296,-0.223342,3.064565,-0.529559,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0
15,55264,0.434386,-0.321296,-0.223342,0.116147,-0.491507,1,0,1,0,...,0,0,1,0,0,1,0,1,0,0


In [116]:
#check if predictors for test set = training data
prediction_features_dropped.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Female', 'gender_Male', 'gender_Other', 'ever_married_No',
       'ever_married_Yes', 'work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed', 'work_type_children',
       'Residence_type_Rural', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

In [115]:
training_features.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Female', 'gender_Male', 'gender_Other', 'ever_married_No',
       'ever_married_Yes', 'work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed', 'work_type_children',
       'Residence_type_Rural', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

In [94]:
#remove index, change order such that it aligns to the training set
prediction_features_dropped = prediction_features_dropped[['age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi', 'gender_Female', 'gender_Male', 'gender_Other', 'ever_married_No',
       'ever_married_Yes', 'work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed', 'work_type_children',
       'Residence_type_Rural', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes']]

#### Just in case you want to create .mat file of cleaned test set.. 

In [ ]:
numpyMatrix_prediction = prediction_features_dropped.as_matrix()
scipy.io.savemat('NAremove_test.mat', {'matrix': numpyMatrix_prediction})

In [ ]:
# RENAME AS PREDICTION_FEATUERES
prediction_features = prediction_features_dropped

In [ ]:
prediction_features.columns
len(prediction_features) 
# add missing categorical feature columns
current_categorical_engineered_features = set(prediction_features.columns) - set(numeric_feature_names)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features

## Create .mat file for test dataset for matlab 

In [ ]:
scipy.io.savemat('NAremove_train.mat', {'matrix': numpyMatrix})
scipy.io.savemat('NAremove_train_target.mat', {'matrix': numpyMatrix_target})
colname = training_features.columns
# Since colomun names are dropped in the .mat file, another file 
# containing those is created 

scipy.io.savemat('NAremove_train_target_names.mat', {'matrix': outcome_name})
scipy.io.savemat('NAremove_train_colnames.mat', {'matrix': colname.values})